# Placemeter

This notebook is focused on how to make a DataFram for Placemeter's sensors all aroudn New York City through their API and plot the results. Thus, It is necessarily to have Placemeter API token.


In [1]:
%load_ext watermark
%watermark -v -m -p pandas,numpy

import json
import codecs

import pandas as pd
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
from datetime import date
from time import mktime

import uncurl
import requests

### A .txt file with the Placemeter API token should be placed with this code file
api_key = open('token.txt', 'r').read() 

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Bar, Scatter, Figure, Layout

from plotly.graph_objs import *
import numpy as np

import cufflinks as cf

print ("Plotly  ", __version__)
init_notebook_mode(connected=True)

import matplotlib.pyplot as plt
%matplotlib inline 


CPython 3.5.2
IPython 4.2.0

pandas 0.18.1
numpy 1.11.1

compiler   : GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)
system     : Darwin
release    : 15.6.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit
Plotly   1.12.5


#### In the API, the following GET command is recommended for downloading the data. The command has to be converted to “requests.get ()” for use in python. 

In [2]:
uncurl.parse ('GET https://api.placemeter.net/api/v1/measurementpoints/id/data?start=start&end=end&resolution=resolution&metrics=metrics[&classes=classes][&include_unreliable_data=include_unreliable_data]')

'requests.get("https://api.placemeter.net/api/v1/measurementpoints/id/data?start=start&end=end&resolution=resolution&metrics=metrics[&classes=classes][&include_unreliable_data=include_unreliable_data]",\n    headers={},\n    cookies={},\n)'

In [3]:
r = requests.get("https://api.placemeter.net/api/v1/measurementpoints", headers={ "Authorization": api_key })
data = r.json()
df1 = pd.DataFrame.from_dict(data)
df1.head()

,classes,id,location,metrics,name,type
0,[all],14331,"{'latitude': 40.7198744, 'longitude': -74.0018...","[{'name': 'Direction 1', 'id': 'direction_1'},...",Broadway @ Canal - Street,turnstile
1,[all],14333,"{'latitude': 40.7198744, 'longitude': -74.0018...","[{'name': 'Direction 1', 'id': 'direction_1'},...",Canal Subway NQR,doorway
2,[all],14332,"{'latitude': 40.7198744, 'longitude': -74.0018...","[{'name': 'Direction 1', 'id': 'direction_1'},...",Broadway @ Canal - West Sidewalk,turnstile
3,[all],17204,"{'latitude': None, 'longitude': None}","[{'name': 'Direction 1', 'id': 'direction_1'},...",Canal South Half?,turnstile
4,[all],17205,"{'latitude': None, 'longitude': None}","[{'name': 'Traffic in', 'id': 'direction_1'}, ...",Area 1,None


# Single sensor 
The following code is to download the data from sensor 14333 and plot the number of recorded people between 2016-05-15 and 2016-08-31.

In [4]:
## 14333
r14333 = requests.get("https://api.placemeter.net/api/v1/measurementpoints/14333/data?start=1463270400&end=1470009600", headers={ "Authorization": api_key })
data14333= r14333.json()
df14333 = pd.DataFrame.from_dict(data14333['data'])

df14333.rename(columns={'all':'colc'}, inplace=True)


df3 = pd.DataFrame(columns= ['direction1', 'direction2'])
for (i,r) in df14333.iterrows():
    e = r['colc']
    df3.loc[i] = [e['direction_1'], e['direction_2']]

df14333 = pd.concat([df14333, df3], axis =1)
df14333 = df14333[['timestamp','unreliable_periods','direction1','direction2']]

df14333[ 'timestamp'] = pd.to_datetime(df14333['timestamp'], unit='s')
df14333 = df14333.set_index( 'timestamp').tz_localize('UTC').tz_convert('America/New_York').tz_convert(None).reset_index()
df14333.set_index( 'timestamp', inplace = True)
df14333.index = df14333.index - DateOffset(hours = 4) 

iplot(df14333.iplot(asFigure=True,
                            kind='scatter',xTitle='Dates',yTitle='Number of People',title='14333'))

# For any number of sensor

In the following I have expanded the pervious code so that we can get the data from more than just one sensor and also for different period of time.   


In [5]:
main_df = pd.DataFrame()

## Sensor IDs
idnumber = '14333', '14331', '14332' 

## Starting day for collecting data.
start = date(2016, 6, 1) 
start =int(mktime(start.timetuple())) 

## End day for collecting data.
end = date(2016, 8, 16) 
end =int(mktime(end.timetuple()))

In [6]:
for sensors in idnumber:
    query  = "https://api.placemeter.net/api/v1/measurementpoints/"+sensors+"/data?start="+str(start)+"&end="+str(end)
    r = requests.get(query, headers={ "Authorization": api_key })
    data= r.json()
    df = pd.DataFrame.from_dict(data['data'])
    df.rename(columns={'unreliable_periods':str(sensors)+'_unreliable_periods', 
                       'timestamp':str(sensors)+'_timestamp',
                       'all':str(sensors)+'_all'}, inplace=True)
    
    df3 = pd.DataFrame(columns= [str(sensors)+'_direction1', str(sensors)+'_direction2'])
    
    for (i,r) in df.iterrows():
        e = r[str(sensors)+'_all']
        df3.loc[i] = [e['direction_1'], e['direction_2']]
    
    df = pd.concat([df, df3], axis =1)
    df = df[[str(sensors)+'_timestamp', str(sensors)+'_unreliable_periods', str(sensors)+'_direction1', str(sensors)+'_direction2']]
    
    df[ str(sensors)+'_timestamp'] = pd.to_datetime(df[ str(sensors)+'_timestamp'], unit='s')
    df = df.set_index( str(sensors)+'_timestamp').tz_localize('UTC').tz_convert('America/New_York').tz_convert(None).reset_index()
    df.set_index( str(sensors)+'_timestamp', inplace = True)
    df.index = df.index - DateOffset(hours = 4) 
    
    if main_df.empty:
        main_df = df
    else:
        main_df = main_df.join(df)

    

#### Unfortunately, Github dose not render iplot yet, so I have added couple of screenshot from the plot and published it on [Anaconda]( https://anaconda.org/sohrabrs/retrieving_data_from-placemeter/notebook)

In [7]:
        
iplot(main_df.iplot(asFigure=True,
                        kind='scatter',xTitle='Dates',yTitle='Number of People',title='All the sensors'))

<img src="fig1.png">
<img src="fig2.png">
